<a href="https://colab.research.google.com/github/jorisschellekens/borb-google-colab-examples/blob/main/using_borb_to_create_a_sudoku_pdf.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# ![borb logo](https://github.com/jorisschellekens/borb/raw/master/logo/borb_64.png) Using `borb` to create a Sudoku PDF

[`borb`](https://github.com/jorisschellekens/borb) is a library for reading, creating and manipulating PDF files in python. borb was created in 2020 by Joris Schellekens and is still in active development. Check out the [GitHub repository](https://github.com/jorisschellekens/borb), or the [borb website](https://borbpdf.com).

In [1]:
pip install borb

     |████████████████████████████████| 6.5 MB 3.5 MB/s 
     |████████████████████████████████| 63 kB 1.4 MB/s 
     |████████████████████████████████| 217 kB 56.2 MB/s 
     |████████████████████████████████| 43 kB 1.2 MB/s 
     |████████████████████████████████| 895 kB 38.8 MB/s 
  Created wheel for qrcode: filename=qrcode-7.3.1-py3-none-any.whl size=40402 sha256=a02b3a75a62a88336dc561863e762c40f90b8e343f42f61e9edfebd69c4835a7
  Stored in directory: /root/.cache/pip/wheels/93/d7/39/a4111be2cfb8e679938aa671a37888b6afb1f9e7d748e94492
Successfully built qrcode
  Attempting uninstall: requests
    Found existing installation: requests 2.23.0
    Uninstalling requests-2.23.0:
      Successfully uninstalled requests-2.23.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-colab 1.0.0 requires requests~=2.23.0, but you have requests 2.27.1 which is incompatib

First you'll define all imports. 
You have a couple of categories of imports:

- basic: Everything needed to create a basic PDF. These are `Document`, `Page` and `PDF`
- layout: Everything borb needs to represent various kinds of content, or perform layout on that content. These are `PageLayout`, `SingleColumnLayout`, `Paragraph` and `Image`
- color: In this example you'll be using borb to automatically create a color-scheme for your `Document`. These classes can help you with representing `Color`: `HexColor`, `HSVColor`, `RGBColor`, `Color`, `Pantone`

It'll become clear over the course of this example why each import is needed.

In [3]:
from borb.pdf.document.document import Document
from borb.pdf.page.page import Page
from borb.pdf.pdf import PDF

from borb.pdf.canvas.layout.page_layout.multi_column_layout import SingleColumnLayout
from borb.pdf.canvas.layout.page_layout.page_layout import PageLayout
from borb.pdf.canvas.layout.text.paragraph import Paragraph
from borb.pdf.canvas.layout.table.flexible_column_width_table import FlexibleColumnWidthTable
from borb.pdf.canvas.layout.table.table import Table, TableCell
from borb.pdf.canvas.layout.image.image import Image
from borb.pdf.canvas.layout.layout_element import Alignment

from borb.pdf.canvas.color.color import HexColor, HSVColor, RGBColor, Color
from borb.pdf.canvas.color.pantone import Pantone

import typing
import random
from decimal import Decimal

Let's set up the basics to build a `Document`. We're going to be using a `PageLayout` to ensure we don't have to calculate the exact position of every `LayoutElement`.

In [4]:
# create new Document
doc: Document = Document()

# create new Page
page: Page = Page()
doc.append_page(page)

# set PageLayout
layout: PageLayout = SingleColumnLayout(page)

Now let's work out a color scheme for the `Document`:

In [5]:
theme_color: Color = HexColor("f1cd2e")
accent_colors: typing.List[Color] = HSVColor.tetradic_rectangle(theme_color)
for i, c in enumerate(accent_colors):
  print("%d %s" % (i, Pantone.find_nearest_pantone_color(c).get_name()))

0 freesia
1 acid-lime
2 palace-blue
3 baja-blue


Next up, you're going to add a logo and title to the `Document`. You're going to explictly state the `width` and `height` of the logo, and specify the `font_color` of the title (using the colors we calculated earlier):

In [6]:
# add logo to the Document
layout.add(Image("https://github.com/jorisschellekens/borb/raw/master/logo/borb.png", width=Decimal(64), height=Decimal(64)))

# add title to the Document
layout.add(Paragraph("borb coffee corner puzzle #%d" % random.randint(1000, 9999), 
                     font_color=theme_color, 
                     font_size=Decimal(20)))

For the purposes of this demo, you'll be representing the sudoku as `str`, each character is either a digit, or `.` (representing an empty cell)

In [7]:
# represent the sudoku as a plaintext str
# every . represents an empty cell in the puzzle
# this is easier to debug/change
sudoku_str: str = """
    3.9...4..
    2..7.9...
    .87......
    75..6.23.
    6..9.4..8
    .28.5..41
    ......59.
    ...1.6..7
    ..6...1.4
    """
sudoku_str = sudoku_str.replace("\t","").replace(" ","").replace("\n","")

You're also going to add a short explanation on how to solve a sudoku, just in case :-)

In [8]:
# add the explanation of how to solve a sudoku
layout.add(Paragraph("""
                Sudoku is a logic-based, combinatorial number-placement puzzle. 
                In classic sudoku, the objective is to fill a 9×9 grid with digits so that each column, each row, 
                and each of the nine 3×3 subgrids that compose the grid contains all of the digits from 1 to 9. 
                The puzzle setter provides a partially completed grid, which for a well-posed puzzle has a single solution.
                """, font="Helvetica-Oblique"))

Now you can finally add the Sudoku to the `Document`. You'll represent the Sudoku as a `Table`. `borb` has two implementations of this abstract class, depending on whether you'd like to have columns with flexible width, or just divide the entire available width among the columns.

Here you'll be using `FlexibleColumnWidthTable`.

In [9]:
# add Table
s: Decimal = Decimal(20)
t: Table = FlexibleColumnWidthTable(number_of_rows=9, number_of_columns=9, horizontal_alignment=Alignment.CENTERED)
for i in range(0, 81):
    r: int = int(i / 9)
    c: int = i % 9
    background_color: Color = HexColor("ffffff")
    if r in [0,1,2,6,7,8] and c in [0,1,2,6,7,8]:
        background_color = accent_colors[3]
    if r in [3,4,5] and c in [3,4,5]:
        background_color = accent_colors[3]
    if sudoku_str[i] == ".":
        t.add(TableCell(Paragraph(" "), preferred_width=s, preferred_height=s, background_color=background_color))
    else:
        t.add(TableCell(Paragraph(sudoku_str[i], text_alignment=Alignment.CENTERED), preferred_width=s, preferred_height=s, background_color=background_color))
t.set_padding_on_all_cells(Decimal(5), Decimal(5), Decimal(5), Decimal(5))
layout.add(t)

With all the hard work done, all that's left is to save the `Document` to disk.

In [10]:
with open("output.pdf", "wb") as pdf_file_handle:
  PDF.dumps(pdf_file_handle, doc)

Congratulations! You should now have a PDF containing a Sudoku.